In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
#Prevent Large File crashing on Jupyter Notebook
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [3]:
asinSet= set()
with open('bookId_Asin.csv', 'r') as f:
    for line in f:
        line = line.strip().split(',')
        asinSet.add(line[1])

In [4]:

l = []
with open('complete_Books.json', 'r') as f:
    with open('amazon_official.tsv', 'w') as g:
        g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+'\t' + 'reviewText' + '\n')
        for i, line in enumerate(f):
            j = json.loads(line)
            if j['asin'] in asinSet:
                if not 'reviewText' in j: 
                    g.write(str(j['overall']) + '\t' + j['reviewTime'].replace(","," ") + '\t'+ j['asin'] + '\t' + '' + '\n' )
                else :
                    #l.append([j['overall'], j['reviewTime'], j['asin'], j['reviewText']])
                    g.write(str(j['overall']) + '\t' + j['reviewTime'].replace(","," ")  + '\t'+ j['asin'] + '\t' + j['reviewText'].replace("\n"," ") + '\n' )
                
            
    print(j.keys())
    print(i)

dict_keys(['overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime'])
51311620


In [5]:
asinDict = {}
textCount = {}
with open('amazon_official.tsv', 'r') as f:
    next(f)
    for i,line in enumerate(f):
        line = line.strip().split('\t')
        overall = float(line[0])
        asin = line[2]
        if not asin in asinDict :
            textCount[asin]=0
            asinDict[asin] =[]
        asinDict[asin].append(overall)
        if len(line) > 3:
            textCount[asin] +=1
        

In [6]:
#Calculate average of each book
average = {}
for asin in asinDict:
    average[asin]=np.round(sum(asinDict[asin])/len(asinDict[asin]),2)

In [7]:
#Number of ratings of each book
rating_count = {}
for asin in asinDict:
    rating_count[asin] = len(asinDict[asin])

In [8]:
#Number of text revirws of each book
text_reviews_count = {}
for asin in asinDict:
    text_reviews_count[asin] = textCount[asin]

In [9]:
result = { asin: [asin,average[asin], rating_count[asin], text_reviews_count[asin] ] for asin in asinDict }

In [10]:
amazon= pd.DataFrame.from_dict(result,orient='index')
amazon.columns = ['asin', 'average', 'rating_count','text_reviews_count']

In [11]:
amazon.reset_index(drop=True)

,asin,average,rating_count,text_reviews_count
0,0001713353,4.83,54,54
1,0001061240,4.87,45,45
2,0001711296,4.44,107,107
3,0002007649,3.37,19,19
4,0001716069,4.61,59,59
...,...,...,...,...
37266,0312943636,4.17,24,24
37267,0312953038,4.00,27,27
37268,0312955154,3.29,14,14
37269,0312956878,3.33,15,15


In [12]:
amazon.to_csv('metadata_amazon.csv', index = False)

# ---------------------------------------
## Merging with genres dataset

In [13]:
genres = pd.read_csv('genres_amazon.csv')
g = pd.DataFrame(genres)
g.columns

Index(['asin', 'genres'], dtype='object')

In [14]:
amazon = pd.read_csv('metadata_amazon.csv')
a = pd.DataFrame(amazon)

In [15]:
meta = pd.merge(a,g, on='asin', how='inner')

In [16]:
meta.isnull().sum()

asin                    0
average                 0
rating_count            0
text_reviews_count      0
genres                846
dtype: int64

In [17]:
new_meta = meta.copy()

In [18]:
new_meta['genres']=new_meta['genres'].astype(str).apply(lambda st: st[st.find("[")+1:st.find("]")]).str.replace('\'', '').str.replace('\"', '')

In [19]:
new_meta['genres'] = new_meta['genres'].str.replace('Books,','',1).str.replace(' Genre','').str.replace(' Arts','Art')
new_meta['genres'] = new_meta['genres'].str.replace(', & Style','',1).str.replace(', ,','')

In [20]:
new_meta.to_csv('amazon_metadata.csv', index = False)

# ----------------------------------------------------------------


In [21]:
### l = []
with open('amazon_official.tsv', 'r') as f:
    with open('amazon_textOnly_official.tsv', 'w') as g:
        #g.write('overall' + '\t' + 'reviewTime' + '\t' + 'asin'+','+'reviewText' + '\n')
        for i, line in enumerate(f):
            line = line.strip().split('\t')
            asin = line[2]
            
            if len(line) > 3:
                text = line[3] 
                g.write(asin + '\t' + text + '\n' )
            else:
                g.write(asin + '\t' +''+ '\n' )